In [1]:
from docplex.mp.model import Model as Model_cpx
import gurobipy as gp
from gurobipy import Model as Model_grb, GRB, quicksum
import math
import time

### Parser que lee el archivo de instancia y retorna un diccionario con los datos.

In [2]:
def parse_file(file_path):
    with open(file_path, 'r') as file:
        lines = [line.strip() for line in file if line.strip()]

    num_customers, num_depots = int(lines[0]), int(lines[1])
    depots = [tuple(map(float, lines[i].split())) for i in range(2, 2 + num_depots)]
    customers = [tuple(map(float, lines[i].split())) for i in range(2 + num_depots, 2 + num_depots + num_customers)]
    vehicle_capacity = float(lines[2 + num_depots + num_customers])
    depot_capacities = list(map(float, lines[3 + num_depots + num_customers:3 + 2 * num_depots + num_customers]))
    customer_demands = list(
        map(float, lines[3 + 2 * num_depots + num_customers:3 + 2 * num_depots + 2 * num_customers]))
    depot_opening_costs = list(
        map(float, lines[3 + 2 * num_depots + 2 * num_customers:3 + 3 * num_depots + 2 * num_customers]))
    route_opening_cost = float(lines[3 + 3 * num_depots + 2 * num_customers])

    all_points = depots + customers
    distance_matrix = [
        [math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2) for x2, y2 in all_points] for x1, y1 in all_points
    ]

    return {
        "num_customers": num_customers,
        "num_depots": num_depots,
        "depots": depots,
        "customers": customers,
        "vehicle_capacity": vehicle_capacity,
        "depot_capacities": depot_capacities,
        "customer_demands": customer_demands,
        "depot_opening_costs": depot_opening_costs,
        "route_opening_cost": route_opening_cost,
        "distance_matrix": distance_matrix,
    }

# Modelo SCF (Single Commodity Flow).

In [3]:
def modelo_scf_cplex(parsed_data):
    """
    Implementación del modelo SCF (Single Commodity Flow) usando CPLEX.

    Parámetros:
        parsed_data (dict): Diccionario con los datos parseados.

    Retorna:
        Model: Modelo de CPLEX con el problema SCF formulado.
    """
    n = parsed_data["num_customers"]
    m = parsed_data["num_depots"]
    depots = parsed_data["depots"]
    customers = parsed_data["customers"]
    vehicle_capacity = parsed_data["vehicle_capacity"]
    depot_capacities = parsed_data["depot_capacities"]
    customer_demands = parsed_data["customer_demands"]
    opening_costs_depots = parsed_data["depot_opening_costs"]
    opening_cost_route = parsed_data["route_opening_cost"]
    costs = parsed_data["distance_matrix"]

    arc_indices = [(i, j) for i in range(m + n) for j in range(m + n) if i != j]

    data = {
        "num_clientes": parsed_data["num_customers"],
        "num_depositos": parsed_data["num_depots"],
        "distancias": {(i, j): parsed_data["distance_matrix"][i][j] for i, j in arc_indices},
        "cap_deposito": parsed_data["depot_capacities"],
        "costo_apertura_deposito": parsed_data["depot_opening_costs"],
        'demanda': parsed_data["customer_demands"],
        'cap_vehiculo': parsed_data["vehicle_capacity"]
    }

    # -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+ Crear modelo +-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-

    modelo = Model_cpx(name="SCF_cplex")

    # -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+ Variables: +-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-

    # x[i, j]: Binaria, indica si se utiliza el arco entre el nodo i y el nodo j
    x = modelo.binary_var_dict(data['distancias'], name="x")
    # f[i, j]: Flujo continuo asociado al arco (i, j)
    f = modelo.continuous_var_dict(data['distancias'], lb=0, name="f")
    # f[d, j]: Flujo de los depósitos hacia los clientes
    fd = modelo.continuous_var_dict([(d, j) for d in range(data['num_depositos']) for j in range(data['num_clientes'])],
                                    lb=0, name="fd")
    # y[d]: Binaria, indica si el depósito d está abierto
    y = modelo.binary_var_list(data['num_depositos'], name="y")
    # -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+ Funcion objetivo: +-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-

    modelo.minimize(
        modelo.sum(data['distancias'][(i, j)] * x[i, j] for (i, j) in data['distancias']) +
        modelo.sum(data['costo_apertura_deposito'][d] * y[d] for d in range(data['num_depositos']))
    )

    # -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+ Restricciones: +-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-

    # 1. El flujo total desde todos los depósitos hacia los clientes debe ser igual al número de clientes
    for d in range(data['num_depositos']):
        modelo.add_constraint(
            modelo.sum(fd[d, j] for j in range(data['num_clientes'])) == data['num_clientes'],
            f"flujo_total_deposito_{d}"
        )

    # 2. Restricción de flujo en los arcos (d, j): El flujo debe estar acotado
    for d in range(data['num_depositos']):
        for j in range(data['num_clientes']):
            modelo.add_constraint(
                fd[d, j] <= data['cap_deposito'][d] * y[d],
                f"flujo_deposito_cliente_{d}_{j}"
            )

    # 3. Restricción de flujo entre clientes y depósitos
    for i, j in data['distancias']:
        if i >= data['num_depositos']:  # Es un cliente
            modelo.add_constraint(
                f[i, j] <= (data['num_clientes'] - 1) * x[i, j],
                f"flujo_cliente_{i}_{j}"
            )
        if j >= data['num_depositos']:  # Es un cliente
            modelo.add_constraint(
                f[i, j] <= (data['num_clientes'] - 1) * x[i, j],
                f"flujo_cliente_{i}_{j}"
            )

    # 4. Restricción de conservación de flujo: Un cliente debe recibir exactamente una unidad de flujo
    for cliente in range(data['num_clientes']):
        modelo.add_constraint(
            modelo.sum(f[i, cliente] for i in range(data['num_depositos'] + data['num_clientes']) if
                       (i, cliente) in data['distancias']) == 1,
            f"conservacion_flujo_cliente_{cliente}"
        )

    # 5. Restricciones para que cada depósito sirva al menos un cliente
    for deposito in range(data['num_depositos']):
        modelo.add_constraint(
            modelo.sum(x[deposito, j] for j in range(data['num_clientes']) if (deposito, j) in data['distancias']) >= y[
                deposito],
            f"deposito_servir_cliente_{deposito}"
        )

    # 6. No debe haber circuitos entre clientes sin pasar por un depósito
    for i, j in data['distancias']:
        if i >= data['num_depositos'] and j >= data['num_depositos']:
            modelo.add_constraint(
                x[i, j] == 0,
                f"no_circuito_sin_deposito_{i}_{j}"
            )

    # 7. No debe haber circuitos con más de un depósito
    for i in range(data['num_depositos']):
        for j in range(data['num_depositos']):
            if i != j:
                modelo.add_constraint(
                    x[i, j] == 0,
                    f"no_circuito_multiples_depositos_{i}_{j}"
                )

    return modelo


In [4]:
def modelo_scf_gurobi(parsed_data):
    """
    Implementación del modelo SCF (Single Commodity Flow) usando Gurobi.

    Parámetros:
        parsed_data (dict): Diccionario con los datos parseados.

    Retorna:
        Model: Modelo de Gurobi con el problema SCF formulado.
    """
    n = parsed_data["num_customers"]
    m = parsed_data["num_depots"]
    depots = parsed_data["depots"]
    customers = parsed_data["customers"]
    vehicle_capacity = parsed_data["vehicle_capacity"]
    depot_capacities = parsed_data["depot_capacities"]
    customer_demands = parsed_data["customer_demands"]
    opening_costs_depots = parsed_data["depot_opening_costs"]
    opening_cost_route = parsed_data["route_opening_cost"]
    costs = parsed_data["distance_matrix"]

    arc_indices = [(i, j) for i in range(m + n) for j in range(m + n) if i != j]

    data = {
        "num_clientes": parsed_data["num_customers"],
        "num_depositos": parsed_data["num_depots"],
        "distancias": {(i, j): parsed_data["distance_matrix"][i][j] for i, j in arc_indices},
        "cap_deposito": parsed_data["depot_capacities"],
        "costo_apertura_deposito": parsed_data["depot_opening_costs"],
        'demanda': parsed_data["customer_demands"],
        'cap_vehiculo': parsed_data["vehicle_capacity"]
    }


    # Crear el modelo
    mdl = Model_grb("SCF_gurobi")

    # Variables
    x = mdl.addVars(data['distancias'], vtype=GRB.BINARY, name="x")  # Ruta entre nodos
    f = mdl.addVars(data['distancias'], lb=0, vtype=GRB.CONTINUOUS, name="f")  # Flujo continuo
    fd = mdl.addVars(
        [(d, j) for d in range(data['num_depositos']) for j in range(data['num_clientes'])],
        lb=0, vtype=GRB.CONTINUOUS, name="fd"
    )  # Flujo desde depósitos a clientes
    y = mdl.addVars(data['num_depositos'], vtype=GRB.BINARY, name="y")  # Uso de depósitos

    # Función objetivo
    mdl.setObjective(
        quicksum(data['distancias'][(i, j)] * x[i, j] for (i, j) in data['distancias']) +
        quicksum(data['costo_apertura_deposito'][d] * y[d] for d in range(data['num_depositos'])),
        GRB.MINIMIZE
    )

    # Restricciones
    # 1. Capacidad de los depósitos
    for deposito in range(data['num_depositos']):
        mdl.addConstr(
            quicksum(
                data['demanda'][j] * x[deposito, j]
                for j in range(data['num_clientes']) if (deposito, j) in data['distancias']
            ) <= data['cap_deposito'][deposito] * y[deposito]
        )

    # 2. Capacidad de los vehículos
    for deposito in range(data['num_depositos']):
        mdl.addConstr(
            quicksum(
                data['demanda'][j] * x[deposito, j]
                for j in range(data['num_clientes']) if (deposito, j) in data['distancias']
            ) <= data['cap_vehiculo'] * y[deposito]
        )

    # 3. Circuitos que comienzan y terminan en el mismo depósito
    for deposito in range(data['num_depositos']):
        mdl.addConstr(
            quicksum(x[deposito, cliente] for cliente in range(data['num_clientes'])
                     if (deposito, cliente) in data['distancias']) ==
            quicksum(x[cliente, deposito] for cliente in range(data['num_clientes'])
                     if (cliente, deposito) in data['distancias'])
        )

    # 4. No debe haber circuitos con más de un depósito
    for i in range(data['num_depositos']):
        for j in range(data['num_depositos']):
            if i != j:
                mdl.addConstr(x[i, j] == 0)

    # 5. No debe haber circuitos entre clientes sin pasar por un depósito
    for i, j in data['distancias']:
        if i >= data['num_depositos'] and j >= data['num_depositos']:
            mdl.addConstr(x[i, j] == 0)

    # 6. Conservación de flujo: Un cliente debe recibir exactamente una unidad de flujo
    for cliente in range(data['num_clientes']):
        mdl.addConstr(
            quicksum(f[i, cliente] for i in range(data['num_depositos'] + data['num_clientes'])
                     if (i, cliente) in data['distancias']) == 1
        )

    # 7. Cada depósito debe servir al menos a un cliente
    for deposito in range(data['num_depositos']):
        mdl.addConstr(
            quicksum(x[deposito, j] for j in range(data['num_clientes'])
                     if (deposito, j) in data['distancias']) >= y[deposito]
        )

    # 8. Flujo total desde todos los depósitos hacia los clientes debe ser igual al número de clientes
    for d in range(data['num_depositos']):
        mdl.addConstr(
            quicksum(fd[d, j] for j in range(data['num_clientes'])) ==
            quicksum(x[d, j] for j in range(data['num_clientes']) if (d, j) in data['distancias'])
        )

    # 9. Restricción de flujo en los arcos (d, j): El flujo debe estar acotado
    for d in range(data['num_depositos']):
        for j in range(data['num_clientes']):
            mdl.addConstr(fd[d, j] <= data['cap_deposito'][d] * y[d])

    # 10. Restricción de flujo entre clientes y depósitos
    for i, j in data['distancias']:
        if i >= data['num_depositos']:  # Es un cliente
            mdl.addConstr(f[i, j] <= (data['num_clientes'] - 1) * x[i, j])
        if j >= data['num_depositos']:  # Es un cliente
            mdl.addConstr(f[i, j] <= (data['num_clientes'] - 1) * x[i, j])

    return mdl

# Modelo CDA (Capacitated Depot Allocation).

In [5]:
def modelo_cda_cplex(parsed_data):
    n = parsed_data["num_customers"]
    m = parsed_data["num_depots"]
    depots = parsed_data["depots"]
    customers = parsed_data["customers"]
    vehicle_capacity = parsed_data["vehicle_capacity"]
    depot_capacities = parsed_data["depot_capacities"]
    customer_demands = parsed_data["customer_demands"]
    opening_costs_depots = parsed_data["depot_opening_costs"]
    costs = parsed_data["distance_matrix"]

    mdl = Model_cpx(name="CDA_cpx")

    # Variables
    v = mdl.binary_var_matrix(n, m, name="v")  # Cliente asignado a depósito
    x = mdl.binary_var_matrix(n + m, n + m, name="x")  # Uso de arcos
    y = mdl.binary_var_list(m, name="y")  # Uso de depósitos

    # Función objetivo
    mdl.minimize(
        mdl.sum(costs[i][j] * x[i, j] for i in range(n + m) for j in range(n + m)) +
        mdl.sum(opening_costs_depots[d] * y[d] for d in range(m))
    )

    # Restricciones
    # 1. Cada cliente asignado a un único depósito
    for i in range(n):
        mdl.add_constraint(mdl.sum(v[i, d] for d in range(m)) == 1)

    # 2. Si un cliente está asignado a un depósito, el depósito debe estar activo
    for i in range(n):
        for d in range(m):
            mdl.add_constraint(v[i, d] <= y[d])

    # 3. Relación entre asignaciones y rutas
    for i in range(n):
        for j in range(n):
            for d in range(m):
                mdl.add_constraint(v[i, d] + x[i + m, j + m] <= v[j, d] + 1)

    # 4. Capacidades de los depósitos
    for d in range(m):
        mdl.add_constraint(
            mdl.sum(customer_demands[i] * v[i, d] for i in range(n)) <= depot_capacities[d]
        )

    # 5. Capacidad del vehículo
    for i, j in [(i, j) for i in range(n + m) for j in range(n + m) if i != j]:
        mdl.add_constraint(x[i, j] <= vehicle_capacity)

    return mdl


In [6]:
def modelo_cda_gurobi(parsed_data):
    # Extract parameters from parsed data
    n = parsed_data["num_customers"]  # Number of customers
    m = parsed_data["num_depots"]  # Number of depots
    depot_coords = parsed_data["depots"]  # List of depot coords (e.g., [(0, 0), (1, 1), ...])
    customer_coords = parsed_data["customers"]  # List of customer coords (e.g., [(0, 0), (1, 1), ...])
    vehicle_capacity = parsed_data["vehicle_capacity"]  # Capacity of a vehicle
    depot_capacities = parsed_data["depot_capacities"]  # List of depot capacities
    customer_demands = parsed_data["customer_demands"]  # List of customer demands
    opening_costs_depots = parsed_data["depot_opening_costs"]  # List of depot opening costs
    route_opening_cost = parsed_data["route_opening_cost"]  # Cost of opening a route
    costs = parsed_data["distance_matrix"]  # Distance matrix

    # listas de índices para depots y customers
    customers = range(n)
    depots = range(n, n + m)

    # Hacer que keys() funcione
    arc_indices = [(i, j) for i in range(m + n) for j in range(m + n) if i != j]

    # Create a Gurobi model
    model = Model_grb("CDA_grb")

    # Variables
    x = model.addVars(arc_indices, vtype=GRB.BINARY,
                      name="x")  # Variable Binaria que indica si el arco de i a j está siendo usado
    v = model.addVars([(i, d) for i in customers for d in depots], vtype=GRB.CONTINUOUS,
                      name="v")  # Variable continua que indica si el cliente está asignado a un depot d
    y = model.addVars(depots, vtype=GRB.BINARY, name="y")  # Variable binaria que indica si el depot d está abierto

    # Funcion objetivo

    model.setObjective(
        gp.quicksum(costs[i][j] * x[i, j] for i, j in arc_indices) +  # Minimizar el costo de la ruta +
        gp.quicksum(opening_costs_depots[d - n] * y[d] for d in depots) +  # El costo del inicio del depot +
        route_opening_cost * gp.quicksum(x[d, i] for d in depots for i in customers),  # El inicio de la ruta.
        GRB.MINIMIZE
    )

    # Restricción CDA
    # Asegura que cada cliente este asignado a un sólo depot
    model.addConstrs((gp.quicksum(v[i, d] for d in depots) == 1 for i in customers), "client_assignment")

    # Asegura que si se ocupa un arco [i,d], el cliente [d,i] o [i,d] estará ahí
    model.addConstrs((x[d, i] <= v[i, d] for d in depots for i in customers), "client_inclusion_outgoing")
    model.addConstrs((x[i, d] <= v[i, d] for d in depots for i in customers), "client_inclusion_incoming")

    # Restriccion que elimina subtours
    model.addConstrs((v[i, d] + x[i, j] <= v[j, d] + 1 for d in depots for i in customers for j in customers if i != j),
                     "path_elimination")

    # Restricciones genéricas
    # Restriccion capacidad depot
    model.addConstrs(
        (gp.quicksum(customer_demands[i] * v[i, d] for i in customers) <= depot_capacities[d - n] * y[d] for d in
         depots),
        "depot_capacity")

    # Restriccion capacidad vehiculo
    model.addConstrs(
        (gp.quicksum(customer_demands[i] * x[i, j] for i in customers for j in customers if i != j) <= vehicle_capacity
         for
         d in depots), "vehicle_capacity")

    # Restriccion depot abierto
    model.addConstrs((v[i, d] <= y[d] for d in depots for i in customers), "depot_open")

# Ejecución de los modelos.

In [7]:
# archivo de resultados
file = open('results.csv', 'w')
file.write(
    "Modelo,Benchmark,Instancia,Número de Variables,Número de Restricciones,Valor Función Objetivo,Tiempo de Cómputo (s)\n")
file.flush()

In [8]:
# DATA
instances = {'B1': ['coord20-5-1.dat', 'coord100-5-3b.dat', 'coord200-10-3b.dat'],
             'B2': ['coordP111112.dat', 'coordP123222.dat', 'coordP133222.dat'],
             'B3': ['coordChrist50.dat', 'coordDas150.dat', 'coordMin134.dat']}

parsed_data = {'B1': [parse_file('Instances/Benchmark_1/' + inst) for inst in instances['B1']],
               'B2': [parse_file('Instances/Benchmark_2/' + inst) for inst in instances['B2']],
               'B3': [parse_file('Instances/Benchmark_3/' + inst) for inst in instances['B3']]}

In [9]:
# Creación de modelos
# modelos = {'B1': {'cplex': {'SCF': [modelo_scf_cplex(data) for data in parsed_data['B1']],
#                             'CDA': [modelo_cda_cplex(data) for data in parsed_data['B1']]},
#                   'gurobi': {'SCF': [modelo_cda_gurobi(data) for data in parsed_data['B1']],
#                              'CDA': [modelo_cda_gurobi(data) for data in parsed_data['B1']]}},
#            'B2': {'cplex': {'SCF': [modelo_scf_cplex(data) for data in parsed_data['B2']],
#                             'CDA': [modelo_cda_cplex(data) for data in parsed_data['B2']]},
#                   'gurobi': {'SCF': [modelo_cda_gurobi(data) for data in parsed_data['B2']],
#                              'CDA': [modelo_cda_gurobi(data) for data in parsed_data['B2']]}},
#            'B3': {'cplex': {'SCF': [modelo_scf_cplex(data) for data in parsed_data['B3']],
#                             'CDA': [modelo_cda_cplex(data) for data in parsed_data['B3']]},
#                   'gurobi': {'SCF': [modelo_cda_gurobi(data) for data in parsed_data['B3']],
#                              'CDA': [modelo_cda_gurobi(data) for data in parsed_data['B3']]}}}

#modelos = {'SCF': {'cplex': [modelo_scf_cplex(data) for B in parsed_data for data in parsed_data[B]],
#                   'gurobi': [modelo_scf_gurobi(data) for B in parsed_data for data in parsed_data[B]]},
#           'CDA': {'cplex': [modelo_cda_cplex(data) for B in parsed_data for data in parsed_data[B]],
#                   'gurobi': [modelo_cda_gurobi(data) for B in parsed_data for data in parsed_data[B]]}}

modelos = {'SCF': {'cplex': {'B1': [modelo_scf_cplex(data) for data in parsed_data['B1']],
                             'B2': [modelo_scf_cplex(data) for data in parsed_data['B2']],
                             'B3': [modelo_scf_cplex(data) for data in parsed_data['B3']]},
                   'gurobi': {'B1': [modelo_scf_gurobi(data) for data in parsed_data['B1']],
                              'B2': [modelo_scf_gurobi(data) for data in parsed_data['B2']],
                              'B3': [modelo_scf_gurobi(data) for data in parsed_data['B3']]}},
           'CDA': {'cplex': {'B1': [modelo_cda_cplex(data) for data in parsed_data['B1']],
                             'B2': [modelo_cda_cplex(data) for data in parsed_data['B2']],
                             'B3': [modelo_cda_cplex(data) for data in parsed_data['B3']]},
                   'gurobi': {'B1': [modelo_cda_gurobi(data) for data in parsed_data['B1']],
                              'B2': [modelo_cda_gurobi(data) for data in parsed_data['B2']],
                              'B3': [modelo_cda_gurobi(data) for data in parsed_data['B3']]}}}

Set parameter Username
Academic license - for non-commercial use only - expires 2025-10-09


KeyboardInterrupt: 

In [ ]:
def get_metrics_cpx(model_instance, benchmark, instance):
    start_time = time.time()
    model_instance.solve()
    end_time = time.time()
    return {
        "Modelo": model_instance.name,
        "Benchmark": benchmark,
        "Instancia": instance,
        "Número de Variables": model_instance.number_of_variables,
        "Número de Restricciones": model_instance.number_of_constraints,
        "Valor Función Objetivo": model_instance.objective_value if model_instance.solution is not None else "N/A",
        "Tiempo de Cómputo (s)": end_time - start_time
    }

In [ ]:
def get_metrics_grb(model_instance, benchmark, instance):
    start_time = time.time()
    model_instance.optimize()
    end_time = time.time()
    return {
        "Modelo": model_instance.getAttr("ModelName"),
        "Benchmark": benchmark,
        "Instancia": instance,
        "Número de Variables": len(model_instance.getVars()),
        "Número de Restricciones": len(model_instance.getConstrs()),
        "Valor Función Objetivo": model_instance.objVal if model_instance.status == GRB.OPTIMAL else "N/A",
        "Tiempo de Cómputo (s)": end_time - start_time
    }

In [ ]:
# Ejecución de modelos
for model in modelos:
    for solver in modelos[model]:
        for B in modelos[model][solver]:
            for i, model_instance in enumerate(modelos[model][solver][B]):
                if solver == 'cplex':
                    metrics = get_metrics_cpx(model_instance, B, instances[B][i])
                elif solver == 'gurobi':
                    metrics = get_metrics_grb(model_instance, B, instances[B][i])

                print(metrics)
                file.write(f"{metrics['Modelo']},{metrics['Benchmark']},{metrics['Instancia']},{metrics['Número de Variables']},"
                           f"{metrics['Número de Restricciones']},{metrics['Valor Función Objetivo']},"
                           f"{metrics['Tiempo de Cómputo (s)']}\n")
                file.flush()